In [ ]:
%load_ext autoreload
%autoreload 2

import os 
os.environ["LOGURU_LEVEL"] = "INFO"

from oxeo.water.models.utils import data2gdf, fetch_water_list, get_all_paths, get_waterbodies
from oxeo.water.testsuite.testsuite import PixelTestWaterBody, TrainingPixelTestWaterBody
from oxeo.water.models.waterbody_predictor import WaterBodyPredictor

In [ ]:
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_HOST = os.environ.get('DB_HOST')

In [ ]:
from sqlalchemy import create_engine
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:5432/geom')
data = fetch_water_list([25906117, -2939785], engine)
gdf = data2gdf(data)

In [ ]:

constellations=["sentinel-2", "landsat-8"]
root_dir="/home/fran/tiles"
all_paths = get_all_paths(gdf, constellations, root_dir)

In [ ]:
waterbodies = get_waterbodies(gdf, constellations, root_dir)

In [ ]:
waterbodies[1].paths[0].mask_path

In [ ]:
# This Test just load waterbody masks from bucket or local. 
# It doesn't perform the prediction.
ptwb = PixelTestWaterBody(y_true_mask_name = "cnn",
                          y_pred_mask_name = "cnn",
                          waterbody=waterbodies[1], 
                          start_date='2019-01-01', 
                          end_date='2019-01-30', 
                          metrics= {"metric_1": lambda x,y: [1]*len(x),
                                    "metric_2": lambda x,y: [2]*len(x)},
                          fs=None)

In [ ]:
ptwb.calculate_metrics()

In [ ]:
# This Test load a waterbody mask for the y_true from bucket or local. 
# It performs a prediction using a given predictor

import gcsfs
fs = gcsfs.GCSFileSystem()
wbp = WaterBodyPredictor(fs, "cnn", revisit_chunk_size=5, 
                         ckpt_path="gs://oxeo-models/semseg/epoch_012.ckpt",
                         batch_size=4,
                         bands=["nir", "red", "green", "blue", "swir1", "swir2"],
                         target_size=1000
                         )

ttwb = TrainingPixelTestWaterBody(predictor=wbp, y_true_mask_name = "cnn",
                          waterbody=waterbodies[1], 
                          start_date='2019-01-01', 
                          end_date='2019-01-30', 
                          metrics= {"metric_1": lambda x,y: [1]*len(x),
                                    "metric_2": lambda x,y: [2]*len(x)},
                          fs=None)

ttwb.calculate_metrics()

In [ ]:
ttwb.calculate_metrics()